In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  %pip install --quiet git+https://github.com/raulonlab/crypto-band-indicators#egg=crypto_band_indicators

import backtrader as bt
from crypto_band_indicators import TickerDataSource, RwaWeightedAverageStrategy, RwaRebalanceStrategy
from crypto_band_indicators.utils.utils import LogColors
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200

# Rainbow Weighted Average Simulator

Usage...

### Variables

In [ ]:
strategy             = "rebalance"    # Select strategy between "weighted_dca" and "rebalance"
ticker_symbol        = "BTCUSDT"      # currently only works with BTCUSDT
start                = '01/01/2022'   # start date of the simulation. Ex: '01/08/2020' or None
end                  = None           # end date of the simulation. Ex: '30/05/2022' or None
initial_cash         = 10000.0        # initial broker cash. Default 10000 usd
min_order_period     = 5              # Minimum period in days to place orders
base_buy_amount      = 100            # Amount purchased in standard DCA
# weighted_multipliers = [0, 0.1, 0.2, 0.35, 0.5, 0.75, 1, 2.5, 3]      # Default order amount multipliers (weighted) for each index
weighted_multipliers = [0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.3, 2.1, 3.4]    # Fibonacci order amount multipliers (weighted) for each index
rebalance_percents   = [10, 20, 30, 40, 50, 60, 70, 80, 90]         # rebalance percentages for each index

### Run Simulator!

In [ ]:

cerebro = bt.Cerebro(stdstats=True, runonce=True)
cerebro.broker.set_coc(True)

# Data sources
ticker_data_source = TickerDataSource()

# Add data feeds to Cerebro
simulation_data_feed = ticker_data_source.to_backtrade_feed(start, end)
cerebro.adddata(simulation_data_feed)

if strategy == "weighted_dca":
    cerebro.addstrategy(RwaWeightedAverageStrategy, base_buy_amount=base_buy_amount,
                    min_order_period=min_order_period, weighted_multipliers=weighted_multipliers, log=True)
elif strategy == "rebalance":
    cerebro.addstrategy(RwaRebalanceStrategy, min_order_period=min_order_period, rebalance_percents=rebalance_percents, log=True)
else:
    error_message = f"Invalid strategy: '{strategy}'"
    print(f"Error: {error_message}")
    exit()

# Add cash to the virtual broker
cerebro.broker.setcash(initial_cash)    # default: 10k

start_portfolio_value = cerebro.broker.getvalue()
cerebro_results = cerebro.run()

end_cash = cerebro.broker.getcash()
end_portfolio_value = cerebro.broker.getvalue()     # Total value in USDT
end_position = cerebro.getbroker().getposition(data=simulation_data_feed)
start_btc_price, end_btc_price = ticker_data_source.get_value_start_end(
    start=start, end=end)

pnl_value = end_portfolio_value - start_portfolio_value
pnl_percent = (pnl_value / start_portfolio_value) * 100
pnl_color = f"{LogColors.FAIL}" if end_portfolio_value < start_portfolio_value else f"{LogColors.OK}"

print("\nSIMULATION RESULT")
print("------------------------")
print(f"{'Started:':<8} {start_portfolio_value:>10.2f} USD (1 BTC = {start_btc_price:.2f} USD)")
print(f"{'Ended:':<8} {end_portfolio_value:>10.2f} USD ({end_position.size:6f} BTC + {end_cash:.2f} USD in cash | 1 BTC = {end_btc_price:.2f} USD)")
print(f"{'PnL:':<8} {pnl_color}{pnl_value:>10.2f} USD ({pnl_percent:.2f}%){LogColors.ENDC}")


### Plot results

In [ ]:
enable_plot_results = True   # Set True to plot the simulation results

if enable_plot_results:
   if len(cerebro_results) > 0:
        cerebro_results[0].plot()